In [1]:
!pip install open3d
!pip install plyfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7

In [3]:
!unzip hough.zip

Streaming output truncated to the last 5000 lines.
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply133.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply134.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply135.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply136.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply137.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply138.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply139.ply  
  inflating: hough/2024-09-02_155419_243301-12_tray-b-4-a_L2_part_4_downsample_10_without_ears_seg_pipes.ply14.ply  
  infl

In [3]:
import torch
import torch.nn as nn
import open3d as o3d
import numpy as np
import os
from torch.utils.data import DataLoader, Dataset
from sklearn.decomposition import PCA
from plyfile import PlyData, PlyElement
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from sklearn.cluster import DBSCAN
import matplotlib.cm as cm
import torch.nn.functional as F
import shutil

In [4]:
def load_ply_files(folder_path):
    ply_files = []
    filenames = []
    for filename in os.listdir(folder_path):
        filepath = os.path.join(folder_path, filename)
        ply = o3d.io.read_point_cloud(filepath)

        ply_files.append(ply)
        filenames.append(filename)

    print(f"Loaded {len(ply_files)} PLY files.")
    return ply_files, filenames


def compute_average_points(ply_files):
    total_points = 0
    num_files = len(ply_files)

    for pcd in ply_files:
        total_points += np.asarray(pcd.points).shape[0]  # Count points in each PLY file

    average_points = total_points / num_files
    return average_points


def hybrid_loss_chamfer_color(reconstructed, original):
    color_loss = F.mse_loss(reconstructed, original)#, reduction='mean')
    return color_loss


class PointCloudDataset(Dataset):
    def __init__(self, ply_files, filenames, num_points=1173):
        self.ply_files = ply_files
        self.filenames = filenames
        self.num_points = num_points

    def __len__(self):
        return len(self.ply_files)

    def __getitem__(self, idx):
        pcd = self.ply_files[idx]
        filename = self.filenames[idx]

        points = np.asarray(pcd.points)
        colors = np.asarray(pcd.colors)

        sampled_points, sampled_colors = self.preprocess_point_cloud(points, colors, self.num_points)

        return torch.tensor(sampled_points, dtype=torch.float32), torch.tensor(sampled_colors, dtype=torch.float32), filename


    def preprocess_point_cloud(self, points, colors, num_points):
        if len(points) < num_points:
            # If there are fewer points, pad with zeros
            padded_points = np.zeros((num_points, 3), dtype=np.float32)
            padded_points[:len(points)] = points

            padded_colors = np.zeros((num_points, 3), dtype=np.float32)
            padded_colors[:len(colors)] = colors

            return padded_points, padded_colors
        else:
            # Sample points if there are enough
            sampled_indices = np.random.choice(len(points), num_points, replace=False)
            sampled_points = points[sampled_indices]
            sampled_colors = colors[sampled_indices]

            return sampled_points, sampled_colors


class PointNetAutoencoder(nn.Module):
    def __init__(self):
        super(PointNetAutoencoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Conv1d(3, 64, 1),           # Input: (batch_size, 6, num_points)
            nn.BatchNorm1d(64),            # Normalize across channels
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),         # Input: (batch_size, 64, num_points)
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 256, 1),        # Input: (batch_size, 128, num_points)
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(4),       # Retain some spatial information
            nn.Conv1d(256, 512, 1),        # Input: (batch_size, 256, 4)
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)        # Compress to global feature
        )

        # Decoder: Hierarchical expansion with BatchNorm
        self.decoder = nn.Sequential(
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.BatchNorm1d(2048),
            nn.ReLU(),
            nn.Linear(2048, 1173 * 3),
        )


    def forward(self, x):
        x = x.transpose(1, 2)  # Change shape to (batch_size, num_features, num_points)
        encoded = self.encoder(x).view(x.size(0), -1)  # Flatten to (batch_size, 512)
        decoded = self.decoder(encoded).view(-1, 1173, 3)  # Reshape to (batch_size, num_points, num_features)
        reconstructed_colors = torch.tanh(decoded) * 0.5 + 0.5  # Rescale to [0, 1] #[:, :, 3:]
        return reconstructed_colors, encoded # torch.cat((reconstructed_points, reconstructed_colors), dim=2)


def train_autoencoder(model, data_loader, optimizer, num_epochs=50, device='cpu'):
    model.train()
    all_coord_errors = []

    for epoch in range(num_epochs):
        for points, colors, filename in data_loader:
            points = points.to(device)
            colors = colors.to(device)
            optimizer.zero_grad()

            # Forward pass through the model
            reconstructed, _ = model(colors)  # Use the point data for the forward pass

            # Compute the loss (assuming you're comparing reconstructed points to original points or reconstructed colors to original colors)
            loss = hybrid_loss_chamfer_color(reconstructed, colors)  # Assuming you want color loss

            # Backpropagation and optimization
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}')

    return all_coord_errors

In [5]:
folder_path = '/content/hough'
ply_files, filenames = load_ply_files(folder_path)
average_points = compute_average_points(ply_files)

print(f"Average number of points: {average_points}")

Loaded 5462 PLY files.
Average number of points: 1173.4804101061882


In [6]:
# Create a dataset and dataloader
dataset = PointCloudDataset(ply_files, filenames)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Initialize model, optimizer
device = torch.device("cuda")
model = PointNetAutoencoder().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Train the autoencoder
all_coord_errors = train_autoencoder(model, data_loader, optimizer, num_epochs=10, device=device)

Epoch [1/10], Loss: 0.07529346644878387
Epoch [2/10], Loss: 0.07829688489437103
Epoch [3/10], Loss: 0.0787537470459938
Epoch [4/10], Loss: 0.0799945592880249
Epoch [5/10], Loss: 0.06924978643655777
Epoch [6/10], Loss: 0.07499130815267563
Epoch [7/10], Loss: 0.07729125022888184
Epoch [8/10], Loss: 0.07944576442241669
Epoch [9/10], Loss: 0.07644076645374298
Epoch [10/10], Loss: 0.08647195249795914


In [7]:
def find_all_error_pointclouds(model, data_loader, device='cpu'):
    model.to(device)  # Move the model to the correct device
    model.eval()  # Set the model to evaluation mode
    errors_list = []  # List to store errors and filenames
    pointclouds_list = []  # List to store point clouds data and colors
    filenames_list = []  # List to store filenames

    with torch.no_grad():  # No gradient computation during evaluation
        for data, colors, filenames in data_loader:
            data = data.to(device)  # Move data to the same device as the model
            original_colors = colors  # Keep track of the original colors

            # Forward pass through the model
            reconstructed, _ = model(original_colors)

            # Compute reconstruction error (using MSE loss)
            errors = F.mse_loss(reconstructed, original_colors, reduction='none')  # Per element loss
            errors = errors.mean(dim=(1, 2))  # Compute mean error per point cloud

            # Save the errors, data, colors, and filenames
            for i in range(len(errors)):
                errors_list.append(errors[i].item())
                points = data[i].cpu().numpy()  # 3D coordinates
                colors = original_colors[i].cpu().numpy()  # Original colors
                pointclouds_list.append((points, colors))  # Store both points and colors together
                filenames_list.append(filenames[i])

    # Combine the lists into a single list of tuples (error, pointcloud_data, color_data, filename)
    pointcloud_info = list(zip(errors_list, pointclouds_list, filenames_list))
    # Sort by error in descending order
    pointcloud_info.sort(reverse=True, key=lambda x: x[0])

    return pointcloud_info


def save_point_cloud_as_ply(points, colors, filename, output_folder="/content"):
    os.makedirs(output_folder, exist_ok=True)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    if colors.max() > 1:
        colors = colors / 255.0

    pcd.colors = o3d.utility.Vector3dVector(colors)

    ply_filename = os.path.join(output_folder, filename)
    o3d.io.write_point_cloud(ply_filename, pcd)
    print(f"Saved point cloud to {ply_filename}")


def save_errors_to_txt(errors_list, filenames_list, output_folder="/content"):
    txt_filename = os.path.join(output_folder, "errors.txt")

    with open(txt_filename, "w") as file:
        for error, filename in zip(errors_list, filenames_list):
            file.write(f"{filename}, {error}\n")

    print(f"Errors saved to {txt_filename}")


def zip_folder(folder_path, zip_filename):
    shutil.make_archive(zip_filename, 'zip', folder_path)
    print(f"Created zip archive: {zip_filename}.zip")


# Finding all errors and getting the pointcloud information
pointcloud_info = find_all_error_pointclouds(model, data_loader, device='cpu')

# Get the top 3 and bottom 3 point clouds based on error
top_3_pointclouds = pointcloud_info[:3]
bottom_3_pointclouds = pointcloud_info[-3:]
selected_pointclouds = top_3_pointclouds + bottom_3_pointclouds

# Save the selected point clouds with original colors
output_folder = "/content/selected_pointclouds_colors"
errors_list = []  # To store errors for the text file
filenames_list = []  # To store filenames for the text file

for idx, (error, pointcloud_data, filename) in enumerate(selected_pointclouds):
    points, colors = pointcloud_data  # Unpack points and colors
    errors_list.append(error)
    filenames_list.append(filename)

    # Save the point cloud with both points and colors
    save_point_cloud_as_ply(points, colors, f"pointcloud_{idx+1}_{filename}", output_folder)

# Save the errors to a text file
save_errors_to_txt(errors_list, filenames_list, output_folder)

# Zip the folder containing the point clouds
zip_folder(output_folder, "/content/selected_pointclouds_colors")

Saved point cloud to /content/selected_pointclouds_colors/pointcloud_1_2024-09-02_161540_243301-13_tray-b-4-d_L2_part_2_downsample_10_without_ears_seg_pipes.ply324.ply
Saved point cloud to /content/selected_pointclouds_colors/pointcloud_2_2024-09-02_161540_243301-13_tray-b-4-d_L2_part_2_downsample_10_without_ears_seg_pipes.ply277.ply
Saved point cloud to /content/selected_pointclouds_colors/pointcloud_3_2024-09-02_161540_243301-13_tray-b-4-d_L2_part_2_downsample_10_without_ears_seg_pipes.ply272.ply
Saved point cloud to /content/selected_pointclouds_colors/pointcloud_4_2024-09-02_155419_243301-12_tray-b-4-f_L2_part_4_downsample_10_without_ears_seg_pipes.ply45.ply
Saved point cloud to /content/selected_pointclouds_colors/pointcloud_5_2024-09-02_155419_243301-12_tray-b-4-f_L2_part_4_downsample_10_without_ears_seg_pipes.ply85.ply
Saved point cloud to /content/selected_pointclouds_colors/pointcloud_6_2024-09-02_155419_243301-12_tray-b-4-f_L2_part_4_downsample_10_without_ears_seg_pipes.ply75